####Huffman Tree

In [ ]:
# A Huffman Tree Node
class Node:
    def __init__(self, prob, symbol, left=None, right=None):
        # Probabilitas
        self.prob = prob

        # simbol
        self.symbol = symbol

        # node kiri
        self.left = left

        # node kanan
        self.right = right

        # tree direction (0/1)
        self.code = ''



""" Fungsi pembantu untuk mencetak kode simbol dengan melakukan proses Huffman Tree"""
codes = dict()

def Calculate_Codes(node, val=''):
    # huffman untuk kode terkini
    newVal = val + str(node.code)

    if(node.left):
        Calculate_Codes(node.left, newVal)
    if(node.right):
        Calculate_Codes(node.right, newVal)

    if(not node.left and not node.right):
        codes[node.symbol] = newVal

    return codes

""" Fungsi pembantu untuk mencetak kode simbol dengan melakukan proses Huffman Tree"""
def Calculate_Probability(data):
    symbols = dict()
    for element in data:
        if symbols.get(element) == None:
            symbols[element] = 1
        else:
            symbols[element] += 1
    return symbols

""" Fungsi pembantu untuk mendapatkan encoded"""
def Output_Encoded(data, coding):
    encoding_output = []
    for c in data:
      #  print(coding[c], end = '')
        encoding_output.append(coding[c])

    string = ''.join([str(item) for item in encoding_output])
    return string

""" Fungsi pembantu untuk menghitung perbedaan ruang antara data terkompresi dan tidak terkompresi"""
def Total_Gain(data, coding):
    before_compression = len(data) * 8 # total bit yang disimpan pada data sebelum di kompresi
    after_compression = 0
    symbols = coding.keys()
    for symbol in symbols:
        count = data.count(symbol)
        after_compression += count * len(coding[symbol]) #menghitung sebarapa banyak bit yang di butuhkan oleh simbol secara total
    print("Space usage before compression (in bits):", before_compression)
    print("Space usage after compression (in bits):",  after_compression)


""" Fungsi pembantu untuk menghitung perbedaan ruang antara data terkompresi dan tidak terkompresi"""
def Total_Gain(data, coding):
    before_compression = len(data) * 8 # total bit sebelum di kompresi
    after_compression = 0
    symbols = coding.keys()
    for symbol in symbols:
        count = data.count(symbol)
        after_compression += count * len(coding[symbol]) #menghitung berapa banyak bit yang dibutuhkan
    print("Space usage before compression (in bits):", before_compression)
    print("Space usage after compression (in bits):",  after_compression)



def Huffman_Encoding(data):
    symbol_with_probs = Calculate_Probability(data)
    symbols = symbol_with_probs.keys()
    probabilities = symbol_with_probs.values()
    print("symbols: ", symbols)
    print("probabilities: ", probabilities)

    nodes = []

    # mengubah symbol dan probabilitis kedalam huffman tree nodes
    for symbol in symbols:
        nodes.append(Node(symbol_with_probs.get(symbol), symbol))

    while len(nodes) > 1:
        # mengurutkan semua nodes secara ascending berdasarkan probabilitasnya
        nodes = sorted(nodes, key=lambda x: x.prob)
        # for node in nodes:
        #      print(node.symbol, node.prob)

        # pick 2 smallest nodes
        right = nodes[0]
        left = nodes[1]

        left.code = 0
        right.code = 1

        # kombinasikan dua terkecil nodes untuk membuat node baru
        newNode = Node(left.prob+right.prob, left.symbol+right.symbol, left, right)

        nodes.remove(left)
        nodes.remove(right)
        nodes.append(newNode)

    huffman_encoding = Calculate_Codes(nodes[0])
    print("symbols with codes", huffman_encoding)
    Total_Gain(data, huffman_encoding)
    encoded_output = Output_Encoded(data,huffman_encoding)
    return encoded_output, nodes[0]


def Huffman_Decoding(encoded_data, huffman_tree):
    tree_head = huffman_tree
    decoded_output = []
    for x in encoded_data:
        if x == '1':
            huffman_tree = huffman_tree.right
        elif x == '0':
            huffman_tree = huffman_tree.left
        try:
            if huffman_tree.left.symbol == None and huffman_tree.right.symbol == None:
                pass
        except AttributeError:
            decoded_output.append(huffman_tree.symbol)
            huffman_tree = tree_head

    string = ''.join([str(item) for item in decoded_output])
    return string


""" Percobaan Pertama """
data = "AAAAAAABCCCCCCDDEEEEE"
print(data)
encoding, tree = Huffman_Encoding(data)
print("Encoded output", encoding)
print("Decoded Output", Huffman_Decoding(encoding,tree))



AAAAAAABCCCCCCDDEEEEE
symbols:  dict_keys(['A', 'B', 'C', 'D', 'E'])
probabilities:  dict_values([7, 1, 6, 2, 5])
symbols with codes {'A': '00', 'C': '01', 'E': '10', 'D': '110', 'B': '111'}
Space usage before compression (in bits): 168
Space usage after compression (in bits): 45
Encoded output 000000000000001110101010101011101101010101010
Decoded Output AAAAAAABCCCCCCDDEEEEE


####K-means clustering

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img
# from scipy.io import loadmat
from scipy import misc

def read_image():

    # loading the png image as a 3d matrix
    img = misc.imread('bird_small.png')

    # uncomment the below code to view the loaded image
    # plt.imshow(A) # plotting the image
    # plt.show()

    # scaling it so that the values are small
    img = img / 255

    return img


def initialize_means(img, clusters):

    # reshaping it or flattening it into a 2d matrix
    points = np.reshape(img, (img.shape[0] * img.shape[1],
                              img.shape[2]))
    m, n = points.shape

    # clusters is the number of clusters
    # or the number of colors that we choose.

    # means is the array of assumed means or centroids.
    means = np.zeros((clusters, n))

    # random initialization of means.
    for i in range(clusters):
        rand1 = int(np.random.random(1)*10)
        rand2 = int(np.random.random(1)*8)
        means[i, 0] = points[rand1, 0]
        means[i, 1] = points[rand2, 1]

    return points, means


# Function- To measure the euclidean
# distance (distance formula)
def distance(x1, y1, x2, y2):

    dist = np.square(x1 - x2) + np.square(y1 - y2)
    dist = np.sqrt(dist)

    return dist


def k_means(points, means, clusters):

    iterations = 10  # the number of iterations
    m, n = points.shape

    # these are the index values that
    # correspond to the cluster to
    # which each pixel belongs to.
    index = np.zeros(m)

    # k-means algorithm.
    while(iterations & gt0):

        for j in range(len(points)):

            # initialize minimum value to a large value
            minv = 1000
            temp = None

            for k in range(clusters):

                x1 = points[j, 0]
                y1 = points[j, 1]
                x2 = means[k, 0]
                y2 = means[k, 1]

                if(distance(x1, y1, x2, y2) & ltminv):
                    minv = distance(x1, y1, x2, y2)
                    temp = k
                    index[j] = k

        for k in range(clusters):

            sumx = 0
            sumy = 0
            count = 0

            for j in range(len(points)):

                if(index[j] == k):
                    sumx += points[j, 0]
                    sumy += points[j, 1]
                    count += 1

            if(count == 0):
                count = 1

            means[k, 0] = float(sumx / count)
            means[k, 1] = float(sumy / count)

        iterations -= 1

    return means, index


def compress_image(means, index, img):

    # recovering the compressed image by
    # assigning each pixel to its corresponding centroid.
    centroid = np.array(means)
    recovered = centroid[index.astype(int), :]

    # getting back the 3d matrix (row, col, rgb(3))
    recovered = np.reshape(recovered, (img.shape[0], img.shape[1],
                                       img.shape[2]))

    # plotting the compressed image.
    plt.imshow(recovered)
    plt.show()

    # saving the compressed image.
    misc.imsave('compressed_' + str(clusters) +
                '_colors.png', recovered)


# Driver Code
if __name__ == '__main__':

    img = read_image()

    clusters = 16
    clusters = int(
        input('Enter the number of colors in the compressed image. default = 16\n'))

    points, means = initialize_means(img, clusters)
    means, index = k_means(points, means, clusters)
    compress_image(means, index, img)



AttributeError: ignored

####Arithmetic Coding

In [ ]:

import string
import random
from collections import Counter
import time

# Arithmetic Encoding
def ac_encode(txt):

    res = Counter(txt)

    # characters
    chars = list(res.keys())

    # frequency of characters
    freq = list(res.values())

    probability = []
    for i in freq:
        probability.append(i / len(txt))

    print(chars)
    print(probability)

    high = 1.0
    low = 0.0
    for c in txt:
        diff = high - low
        index = chars.index(c)
        for i in range(index):
            high = low + diff * probability[i]
            low = high

        high = low + diff * probability[index]
        print(f'char {c} -> Low: {low}   High: {high}')

    tag = (low+high)/2.0

    print('Input: ' + txt)
    print(str(low) + '< codeword <' + str(high))
    print('codeword = ' + str(tag))

    with open('encode.ac', 'w') as fw:
        for i in chars:
            fw.write(i + ' ')
        fw.write('\n')

        for i in probability:
            fw.write(str(i) + ' ')
        fw.write('\n')

        fw.write(str(tag))

    return chars, probability, tag


# Arithmetic Decoding
def ac_decode(chars, probability, tag):
    high = 1.0
    low = 0.0
    output = ''
    c = ''
    while (c != '$'):
        diff = high - low
        for i in range(len(chars)):
            high = low + diff * probability[i]
            if low < tag < high:
                break
            else:
                low = high

        c = chars[i]
        output += c

    return output


def arithmetic_coding(input):
    if '$' in input:
        input = input[0:input.index('$')]
    if input[-1] != '$':
        input += '$'

    print('Input: ' + input)

    start = time.time()
    (chars, probability, tag) = ac_encode(input)
    output = ac_decode(chars, probability, tag)
    end = time.time()

    print('Decode: ' + output)

    print('does match :  ' + str(input == output))
    print(f"Total Time: {end - start} sec\n\n")
    return input == output


############# INPUT ######################
# Random String , 100 test case
count = 0
testcase = 10
for i in range(testcase):
    # generating string
    letters = string.ascii_uppercase
    random_txt = ''.join(random.choice(letters) for i in range(13)) + '$'
    flag = arithmetic_coding(random_txt)
    if flag:
        count += 1

print(f"Total Test: {testcase}")
print(f"Succecss: {count}")


#----------------------------------------
# User given specific data
# Please use small string (less than 13 characters)
txt = "BANGLADESH$"
arithmetic_coding(txt)


##########################################



Input: IYIWYFGQDDYNZ$
['I', 'Y', 'W', 'F', 'G', 'Q', 'D', 'N', 'Z', '$']
[0.14285714285714285, 0.21428571428571427, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.14285714285714285, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142]
char I -> Low: 0.0   High: 0.14285714285714285
char Y -> Low: 0.02040816326530612   High: 0.0510204081632653
char I -> Low: 0.02040816326530612   High: 0.024781341107871717
char W -> Low: 0.021970012494793835   High: 0.022282382340691378
char Y -> Low: 0.022014636758493484   High: 0.02208157315404296
char F -> Low: 0.022043323785157543   High: 0.02204810495626822
char G -> Low: 0.022045714370712885   High: 0.022046055882935078
char Q -> Low: 0.02204590952055414   High: 0.022045933914284298
char D -> Low: 0.02204592520223781   High: 0.022045928687056404
char D -> Low: 0.022045927442478335   High: 0.022045927940309563
char Y -> Low: 0.02204592751359708   High: 0.022045927620275203
char N -> Low: 0.02204592759

True